# 无目标的激光雷达与可见光相机标定方法

## 一、介绍
无目标环境下(Targetless Environment)的标定方法，旨在在没有标定物的情况下估计$T_{Lidar \rightarrow Camera}$外参。无目标标定的关键是提取激光雷达和可见光相机之间的共同信息，边缘是最常用来作为共同信息的特征。

无目标标定必要性: 由于标定板一般离传感器比较近，外参误差可能在大范围场景中扩大，导致远距离点云着色出现偏差。此外，并不是所有的任务都能在一开始用目标进行标定。综合以上两点，研究无目标环境下的标定算法是非常有必要的。

## 二、文献1
文献[1]提出了一种在无目标环境下逐像素标定外参的方法。系统的原理是**提取图像和点云中的天然边缘特征，并且最小化重投影误差**。

文献有以下三点贡献：
* 分析边缘特征提供的约束数量，边缘分布对标定结果的影响。
* 基于激光雷达测量原理，分析常用的基于深度不连续（depth-discontinuous）的点云边缘的弊端，提出深度连续(depth-continuous)的边缘提取方法。
* 不同室内外场景验证方法的有效性，堪比target-based方法。

### 2.1 边缘约束
激光雷达和图像的一对匹配边缘可以提供两个约束。如下图，蓝色边为点云中3D边缘，红色边为图像中对应的2D边缘。蓝色边缘沿着C,D方向的平移和A,B方向的旋转不会改变投影结果，所以自由度为4,提供2个约束。

<p align="center">
    <img src="../resources/paper-targetless/paper1-edgeconstraint.png" width=30%>
</p>

### 2.2 提取边缘特征
#### 2.2.1 2D提取边缘
对于点云**边缘提取**，一种思路先将点云投影到图像平面再提取特征，这种思路会因为遮挡而导致多值(multi-valued)和零值(zero-valued)投影现象。

如下图，A区域相机可见，雷达不可见，故没有投影点，产生零值现象。B区域激光雷达可见，相机不可见，B区域背景点会投影到前景目标（黑点）处，造成多值现象。

<p align="center">
    <img src="../resources/paper-targetless/paper1-project-drawback.png" width=30%>
</p>

#### 2.2.2  3D提取边缘
另一种思路，是直接在点云上提取边缘。有两种边缘：深度不连续的(depth-discontinuous)，深度连续的(depth-continuous)。如下图，深度连续边缘指平面相交处边缘，其深度连续变化。深度不连续边缘则指前景、背景之间深度跃变的边缘。
<p align="center">
    <img src="../resources/paper-targetless/paper1-edge.png" width=30%>
</p>

深度不连续边缘不可靠。理由如下：由于一个激光脉冲并不是一个理想的点，而是有一定发散角度的光束，当从前景目标扫描到背景目标时，一部分激光脉冲被前景目标反射，一部分被背景目标反射，会产生**两个反射脉冲**。当前景反射强度大，第一个反射脉冲占主导地位，即使光束中心线偏移了前景目标，这仍会导致超出前景目标边缘的假目标点。（如下图A处最左侧黄点）。当前景目标与背景比较靠近，两个脉冲会合并，会产生连接前后景目标的点。（称为bleeding points, A处黄点）这两种现象会使前景膨胀，造成**边缘提取误差**。
<p align="center">
    <img src="../resources/paper-targetless/paper1-bleedingpoints.png" width=30%>
</p>

因此，本文提取深度连续的点边缘，分三步：
1）点云体素化 ,体素大小是室外1m，室内0.5m。
2）在体素中，使用RANSAC拟合提取平面。
3）保存互相连接，且角度在30-150度的平面对。平面相交提取边缘。
<p align="center">
    <img src="../resources/paper-targetless/paper1-edge-extraction.png" width=30%>
</p>

对于图像边缘提取，使用Canny算子。提取的边缘像素点保存在k-D树(k=2)中，用于匹配。

### 2.3 匹配
先对点云的每个边缘线进行采样，得到若干3D边缘点。依次进行外参变换、针孔相机投影、畸变矫正操作得到对应的2D像素。然后在图像边缘构造的KD树中，搜索k近邻。求平均计算对应中心点$\vec{q}$，利用协方差矩阵最小特征值对应特征向量作为法向量$\vec{n}$。则$(\vec{q}, \vec{n})$就是图像边缘的**参数表示**。利用**投影点在线上**和**投影线的朝向与$\vec{n}$的正交关系**构造优化目标函数，进行外参优化。

该无目标标定方法类似ICP算法——每次迭代先通过找k近邻确定匹配关系(correspondance)，再构造损失函数进行优化。不同的是该算法的损失函数是非线性的，且没有闭式解，因此算法收敛较慢。在阅读源代码时，发现该算法使用了启发式的优化方法，会在优化过程中逐步减少距离阈值(用越来越少的匹配点构造优化目标函数)。

<p align="center">
    <img src="../resources/paper-targetless/paper1-src_code.png" width=30%>
</p>


此外，和ICP算法一样，当初始位姿不太好时，算法迭代容易收敛到局部极小点，因此文献[1]设计了一个搜索策略进行初始位姿的估计。

<p align="center">
    <img src="../resources/paper-targetless/paper1-initial_value.png" width=30%>
</p>


## 二、文献2
### 2.1 介绍
论文[2]CamVox提出了一种激光雷达结合ORB-SLAM2的SLAM方法，并针对非重复扫描激光雷达和可见光相机提出了一种自动化、无目标的外参标定方法，整体流程图如下所示。


<p align="center">
    <img src="../resources/paper-targetless/paper2-pipeline.png" width=30%>
</p>

1. 由于Livox Horizon非重复扫描的特性，随时间积分获得稠密点云。
2. 根据初始外参投影到图像上，获得一张高分辨率的深度图片和反射率图片。
3. 对所有图像进行直方图均衡化，并用Canny算子进行图像边缘提取，将深度图和反射率图中的边缘叠加在一起。
4. 为获得可靠边缘，对图像边缘进行滤波，滤除长度小于200像素的边缘和交叠在一起的内部边缘。
5. 接着和文献[1]类似通过构造点到线距离的目标函数进行坐标下降优化。


## 参考文献
[1] C. Yuan, X. Liu, X. Hong and F. Zhang, "Pixel-Level Extrinsic Self Calibration of High Resolution LiDAR and Camera in Targetless Environments," in IEEE Robotics and Automation Letters, vol. 6, no. 4, pp. 7517-7524, Oct. 2021, doi: 10.1109/LRA.2021.3098923.

[2] [CSDN-文献1](https://blog.csdn.net/muyiyushan/article/details/118573929)

[3] Zhu Y , Zheng C , Yuan C , et al. CamVox: A Low-cost and Accurate Lidar-assisted Visual SLAM System[J]. 2020.